In [1]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在计算y关于x的梯度之前,我们需要一个地方来存储梯度

In [2]:
x.requires_grad_(True)  # 等价于 x=torch.arange(4.0,requires)
x.grad # 默认值是None

In [3]:
y = 2 * torch.dot(x,x)
y # grad_fn为求梯度的函数

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算y关于x每个分量的梯度

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

计算x的另一个函数

In [9]:
# 默认情况下，pytorch自动累积梯度，我们需要清楚之前的值
x.grad.zero_() # _表示重写内容
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

我们的目的不是计算微分矩阵,而是批量中每个样本单独计算的偏导数之和

In [10]:
# 对非标量调用backward需要传入一个gradient参数，改参数指定微分函数
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [12]:
x.grad.zero_() # 清零梯度
y = x * x
u = y.detach() # u是一个常数 detach返回一个新的tensor，从当前计算图中分离下来的，但是仍指向原变量的存放位置
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [13]:
x.grad.zero_() # 清零梯度
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过python控制流(条件,循环,或者任意函数调用),我们仍然可以计算得到的变量的梯度

In [15]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(),requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)

深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上，然后记录目标值的计算，执行它的反向传播函数，并访问得到的梯度.

requires_grad=True 的作用是让 backward 可以追踪这个参数并且计算它的梯度,只有 requires_grad=True 的参数才会参与求导